In [8]:
import pymysql
import requests

### 下載分析pm2.5 opendata 資料

In [11]:
url="https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON"

In [12]:
resp=requests.get(url,verify=False)
resp

c:\Users\USER\Desktop\shuan\MySQL\pm25_tomysql\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [13]:
resp.json()

{'fields': [{'id': 'site', 'type': 'text', 'info': {'label': '測站名稱'}},
  {'id': 'county', 'type': 'text', 'info': {'label': '縣市名稱'}},
  {'id': 'pm25', 'type': 'text', 'info': {'label': '細懸浮微粒濃度'}},
  {'id': 'datacreationdate', 'type': 'text', 'info': {'label': '資料建置日期'}},
  {'id': 'itemunit', 'type': 'text', 'info': {'label': '測項單位'}}],
 'resource_id': 'c1f31192-babd-4105-b880-a4c2e23a3276',
 '__extras': {'api_key': '540e2ca4-41e1-4186-8497-fdd67024ac44'},
 'include_total': True,
 'total': '1289626',
 'resource_format': 'object',
 'limit': '1000',
 'offset': '0',
 '_links': {'start': '/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&format=JSON&sort=datacreationdate desc',
  'next': '/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&format=JSON&sort=datacreationdate desc&offset=1000'},
 'records': [{'site': '員林',
   'county': '彰化縣',
   'pm25': '4',
   'datacreationdate': '2025-08-20 19:00',
   'itemunit': 'μg/m3'},
  {'site': '大城',
   'cou

In [15]:
datas=resp.json()['records']
datas

[{'site': '員林',
  'county': '彰化縣',
  'pm25': '4',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '大城',
  'county': '彰化縣',
  'pm25': '5',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '富貴角',
  'county': '新北市',
  'pm25': '5',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '麥寮',
  'county': '雲林縣',
  'pm25': '1',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '關山',
  'county': '臺東縣',
  'pm25': '4',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '馬公',
  'county': '澎湖縣',
  'pm25': '2',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '金門',
  'county': '金門縣',
  'pm25': '',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '馬祖',
  'county': '連江縣',
  'pm25': '13',
  'datacreationdate': '2025-08-20 19:00',
  'itemunit': 'μg/m3'},
 {'site': '埔里',
  'county': '南投縣',
  'pm25': '2',
  'datacreationdate':

In [16]:
len(datas)

1000

In [17]:
datas[0]

{'site': '員林',
 'county': '彰化縣',
 'pm25': '4',
 'datacreationdate': '2025-08-20 19:00',
 'itemunit': 'μg/m3'}

### 建立資料表

In [47]:
table_str="""
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)
"""
print(table_str)



create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)



### 連接資料庫

In [71]:
conn=pymysql.connect(
    host="localhost",
    # host="127.0.0.1",
    user="root",
    password="",
    port=3307,
    database="demo"
)

cursor=conn.cursor()
cursor

In [49]:
cursor.execute(table_str)
conn.commit()

### 寫入一筆資料

In [56]:
sqlstr="insert ignore into pm25 (site,county,pm25,datacreationdate,itemunit)\
      values('{}','{}',{},'{}','{}')"

data=list(datas[0].values())
print(data)
sqlstr.format(data[0],data[1],data[2],data[3],data[4])


['員林', '彰化縣', '4', '2025-08-20 19:00', 'μg/m3']


"insert ignore into pm25 (site,county,pm25,datacreationdate,itemunit)      values('員林','彰化縣',4,'2025-08-20 19:00','μg/m3')"

In [58]:
cursor.execute(sqlstr.format(data[0],data[1],data[2],data[3],data[4]))
conn.commit()

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near '%s,%s,%s,%s,%s)' at line 1")

### 一次寫入
 - 加上 ignore

In [64]:
sqlstr="insert ignore into pm25(site,county,pm25,datacreationdate,itemunit)\
      values(%s,%s,%s,%s,%s)"

# 移除pm25空字串部分
values=[list(data.values()) for data in datas if list(data.values())[2]!=""]
# values=[list(data.values()) for data in datas]

#values

cursor.executemany(sqlstr,values)
conn.commit()

In [59]:
values

[['員林', '彰化縣', '4', '2025-08-20 19:00', 'μg/m3'],
 ['大城', '彰化縣', '5', '2025-08-20 19:00', 'μg/m3'],
 ['富貴角', '新北市', '5', '2025-08-20 19:00', 'μg/m3'],
 ['麥寮', '雲林縣', '1', '2025-08-20 19:00', 'μg/m3'],
 ['關山', '臺東縣', '4', '2025-08-20 19:00', 'μg/m3'],
 ['馬公', '澎湖縣', '2', '2025-08-20 19:00', 'μg/m3'],
 ['馬祖', '連江縣', '13', '2025-08-20 19:00', 'μg/m3'],
 ['埔里', '南投縣', '2', '2025-08-20 19:00', 'μg/m3'],
 ['復興', '高雄市', '2', '2025-08-20 19:00', 'μg/m3'],
 ['永和', '新北市', '8', '2025-08-20 19:00', 'μg/m3'],
 ['竹山', '南投縣', '2', '2025-08-20 19:00', 'μg/m3'],
 ['中壢', '桃園市', '10', '2025-08-20 19:00', 'μg/m3'],
 ['三重', '新北市', '11', '2025-08-20 19:00', 'μg/m3'],
 ['冬山', '宜蘭縣', '9', '2025-08-20 19:00', 'μg/m3'],
 ['宜蘭', '宜蘭縣', '5', '2025-08-20 19:00', 'μg/m3'],
 ['陽明', '臺北市', '3', '2025-08-20 19:00', 'μg/m3'],
 ['花蓮', '花蓮縣', '6', '2025-08-20 19:00', 'μg/m3'],
 ['臺東', '臺東縣', '10', '2025-08-20 19:00', 'μg/m3'],
 ['恆春', '屏東縣', '5', '2025-08-20 19:00', 'μg/m3'],
 ['潮州', '屏東縣', '13', '2025-08-20 19:00', 'μg/

In [67]:
conn.close()

Error: Already closed

In [68]:
import pandas as pd

In [72]:
cursor.execute("select * from pm25")

datas=cursor.fetchall()

datas

((1, '員林', '彰化縣', 4, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (4424, '大城', '彰化縣', 5, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (4425, '富貴角', '新北市', 5, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (4426, '麥寮', '雲林縣', 1, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (4427, '關山', '臺東縣', 4, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (4428, '馬公', '澎湖縣', 2, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (4429, '馬祖', '連江縣', 13, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (4430, '埔里', '南投縣', 2, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (4431, '復興', '高雄市', 2, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (4432, '永和', '新北市', 8, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (4433, '竹山', '南投縣', 2, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (4434, '中壢', '桃園市', 10, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (4435, '三重', '新北市', 11, datetime.datetime(2025, 8, 20, 19, 0), 'μg/m3'),
 (4436, '冬山', '宜蘭縣', 9, datetime.datetime(2025, 8,

In [74]:
df=pd.DataFrame(datas,columns=["id","site","county","pm25","datetime","unit"])
df

,id,site,county,pm25,datetime,unit
0,1,員林,彰化縣,4,2025-08-20 19:00:00,μg/m3
1,4424,大城,彰化縣,5,2025-08-20 19:00:00,μg/m3
2,4425,富貴角,新北市,5,2025-08-20 19:00:00,μg/m3
3,4426,麥寮,雲林縣,1,2025-08-20 19:00:00,μg/m3
4,4427,關山,臺東縣,4,2025-08-20 19:00:00,μg/m3
...,...,...,...,...,...,...
476,4899,新莊,新北市,12,2025-08-20 13:00:00,μg/m3
477,4900,板橋,新北市,10,2025-08-20 13:00:00,μg/m3
478,4901,土城,新北市,12,2025-08-20 13:00:00,μg/m3
479,4902,汐止,新北市,7,2025-08-20 13:00:00,μg/m3


In [75]:
df[["site","datetime"]].drop_duplicates()

,site,datetime
0,員林,2025-08-20 19:00:00
1,大城,2025-08-20 19:00:00
2,富貴角,2025-08-20 19:00:00
3,麥寮,2025-08-20 19:00:00
4,關山,2025-08-20 19:00:00
...,...,...
476,新莊,2025-08-20 13:00:00
477,板橋,2025-08-20 13:00:00
478,土城,2025-08-20 13:00:00
479,汐止,2025-08-20 13:00:00
